# Geometry Documentation
---

#### Contents
- 1 Organization
  - 1.1 Design Objects
- 2 Convention
- 3 Design Object Classes
  - 3.1 User Functions
  - 3.2 All Functions
---

## 1 Organization
The geometry creation file `Aircraft_Geometry.py` contains the `Update_Geometry()` function. The format follows .avl file format; information is broken down into the header, SURFACEs, SECTIONs, and CONTROLs. For ease of converting planform parameters into AVL-equivalents, the first few lines provide room to insert planform information. The user defines their header, SURFACEs, SECTIONs, and CONTROLs, and the AVL formatting is done automatically.

#### 1.1 Design Objects
ADaX uses classes to organize each section of the .avl file. There are four classes: **des_plane**, **des_surf**, **des_sec**, and **des_ctrl**. These share the **des_obj** superclass. 

- Header infromation is stored in **des_plane**

- SURFACE information is stored in **des_surf**

- SECTION information is stored in **des_sec**

- CONTROL information is stored in **des_ctrl**

---
## 2 Convention
When naming variables in `Update_Geometry()`, include a three-digit number in the name (e.g., wing_section000, wing_section100, wing_section060). This is similar to G-code convention, where you leave space in between major steps to insert intermediate steps. This is not a major consideration for modern programming IDEs, but is more important for old fixed-format programming languages. 

You will find it useful to make 000 your root and 100 as your tip. If you were to make 1 your root and 2 your tip, then it does not make sense for 3 to go in-between. When you are defining upwards of ten sections in your wing, you may lose track of where sections 4 or 7 go. However, if you use the three-digit designation, it is intuitive where sections 043 and 077 are located relative to the root and tip. 


---
## 3 Design Object Classes
This section covers the design object subclasses in full using examples. Section 3.1 covers only functions to be used by standard users, but Section 3.2 contains every function in the class.

#### 3.1 User Functions
This is for teaching first-time users of the commands they'll be using in the geometry creation file.

- **des_obj**
  - `modify()`
  - `assign()`

- **des_plane**
  - `use_reference()`

- **des_sec**
  - `interp()`

To begin, each design object has a constructor that requires a parent. The parent of a des_surf is a des_plane, the parent of a des_sec is a des_surf, and the parent of a des_ctrl is a des_sec. des_plane has no parents and des_ctrl has no children.

---
```.
plane = des_plane()

wing = des_surf(parent=plane)

wingsec000 = des_sec(parent=wing)
wingsec100 = des_sec(parent=wing)

aileron000 = des_ctrl(parent=wingsec000)
aileron100 = des_ctrl(parent=wingsec100)
```
---

To add information about the header, SURFACE, SECTION, or CONTROL, you have three options:
1. Add the information on object definition using keyword-value pairs
2. Add information after object definition using keyword-value pairs in `.modify()`  
3. Add information after object definition using keyword-value pairs in `.assign()` 

---
```.
#Option 1
plane = des_plane(Sref=5000.0)

#Option 2
plane = des_plane()
plane.assign(Sref=5000.0)

#Option 3
plane = des_plane()
plane.modify(Sref=5000.0)
```
---

All three yield the same result. Although the **des_obj** functions `modify()` and `assign()` are only listed under **des_obj**, they are available to all design objects (i.e., you can use them on des_plane, des_surf, des_sec, and des_ctrl objects). However, there are some slight differences between the three.

1. Object definition can only assign "standard information" to an object. This is the minimum or typical information that is assigned to the object, and is determined in the source code. 

2. `.assign()` can be used to add any AVL keyword to an object. This is done to assign "non-standard information" to an object". No check is performed for the keyword provided and no error is raised.

3. `.modify()` is used to modify existing parameters of the object, and checks to see if the keyword is part of the design object. Once a parameter is assigned using `.assign()`, it may be modified using `.modify()`.

For the first method, here is a table of standard information accepted by object definition and `.modify()`. The keywords are case-sensitive.

| **Object** | **Standard information** | 
|-|-|
|des_plane|Name, Mach, cref, bref, Sref, Xref, Yref, Zref, CDo|
|des_surf|Name, Nchordwise, Cspace, Nspanwise, Sspace, YDUPLICATE, TRANSLATE, CDCL|
|des_sec|Xle, Yle, Zle, Chord, Ainc, Nspanwise, Sspace, AFILE|
|des_ctrl|Cname, Cgain, Xhinge, HingeVec, SgnDup| 

The **des_plane** function `.use_reference()` is used to assign a **des_surf** as the reference surface. If a reference surface is applied, the user-specified *cref*, *bref*, and *Sref* will be overwritten in **des_plane**. 

You will likely have the reference values on hand for the first iteration of your plane, but as you test changes to the wing geometry you might be interested in having ADaX do the calculations for you.

---
```.
plane = des_plane(Name='TitanWing', Mach=0.8, Sref = 0, bref = 0, cref = 0, Xref=45.478, Yref=0.0, Zref=4.83)
wing = des_surf(plane, Name='Wing', YDUPLICATE=0.0, Nspanwise=71)
plane.use_reference(wing)

#This is directly from the code I used in senior design
```
---

The **des_sec** function `.interp()` uses linear interpolation to create a new section in-between two existing sections. `.interp()` is good for creating sections where CONTROLs need to be or for fine-tuning your planform. Instead of manually entering all the information for your control surface start and end locations, you may use `.interp()` to create those sections. You may also choose to create an interpolated section then use `.modify()` to adjust the parameters for that section.

---
```.
hstabsec000 = des_sec(hstab, Xle=0.0, Yle=0.0, Zle=0.0, Chord=28.8, AFILE=af2)
hstabsec100 = des_sec(hstab, Xle=31.50934, Yle=45, Zle=0.0, Chord=7.2, AFILE=af2)

hstabsec010 = des_sec.interp(s1=hstabsec000, s2=hstabsec100, span=2.5)
hstabsec090 = des_sec.interp(s1=hstabsec000, s2=hstabsec100, span=40.5)

elvstart = des_ctrl(hstabsec010, Cname='Elevator', Xhinge=0.80, SgnDup=1)
elvend   = des_ctrl(hstabsec090, Cname='Elevator', Xhinge=0.80, SgnDup=1)

#This is also directly from the code I used in senior design
```
---

#### 3.2 All Functions
This is primarily for those interested in adding to the existing library. It is also for those who are diabolically bored. If you are neither of these, this is the end of this document.

- **des_obj**
  - `modify()`
  - `assign()`
  - `__assign()`
  - `__lv_pair()`
  - `config_to_avl()`
  - `children()`
  - `parents()`
  - **des_plane**
    - `write_to_avl()`
    - `use_reference()`
  - **des_surf**
    - `calculate_reference_values()`
    - `order()`
  - **des_sec**
    - `interp()`

Let's take it from the top.

- `des_obj.modify()`: Modifies an existing value in the design object's .config dictionary. Takes keyword-value pairs as arguments. Notifies the user if the keyword does not exist in the .config dictionary. Returns nothing. 
- `des_obj.assign()`: Assigns a new value to or overwrites existing values in the .config dictionary. Takes keyword-value pairs as arguments. Returns nothing.
- `des_obj.__assign()`: Private method used for assigning values at object definition. Takes keyword-value pairs as arguments. Notifies the user if they keyword is not in the standard information. Returns nothing.
- `des_obj.__lv_pair()`: Private method for writing "label-value" pairs to the .avl file. Uses the formatting keys in the design object to control newline printing for AVL keywords versus non-keyworded values. Takes key-item pairs in dictionary form and a boolean to control commenting. Returns nothing.
- `des_obj.config_to_avl()`: Parses the design object .config and formatting information to call `__lv_pair()` automatically. Called for each design object when .avl file is written. Takes a string containing the .avl file contents. Returns nothing. 
- `des_obj.children()`: Prints the children of the design object. Takes nothing. Returns nothing.
- `des_obj.parents()`: Prints the parents of the design object. Takes nothing. Returns nothing.
- `des_plane.write_to_avl()`: Begins writing the design plane and all of its children to a .avl file. Used at the end of the geometry creation file. Belongs to the design plane object as the function calls `config_to_avl()` for each design object, looping through the child design surfaces, design sections, and design controls. Takes a filename as a string; assumed directory is "AVL/Planes". Returns nothing.
- `des_plane.use_reference()`: Assigns a des_surf as the reference surface for the model. If used, the surface's span, mean aerodynamic chord, and area will be used as the header *bref*, *cref*, and *Sref* reference values. If not used, the reference values must be manually inputted. Takes a **des_surf** object. Returns nothing.
- `des_surf.calculate_reference_values()`: Calculates the span, mean aerodynamic chord, and area of the surface. Stores the results in *des_surf.cref*, *des_surf.bref*, and *des_surf.Sref*
- `des_surf.order()`: Takes the children of the **des_surf** object and orders them based on distance from the root. This is done to make sure the sections are written in order. Takes an integer input 0 for horizontal ordering (ordered by Yle) and 1 for vertical ordering (ordered by Zle). Returns nothing.
- `des_sec.interp()`: Creates a new **des_sec** object using linear interpolation between two existing sections. Typically used for creating sections where control surfaces must be defined. Takes two existing **des_sec** objects, a float spanwise location, and integer orientation; orientation 0 is horizontal and uses Yle (span) as the independent variable, orientation 1 is vertical and uses Zle (height) as the independent variable.
